## 오라클26ai 하이브리드 검색 데모

In [1]:
# 23ai 접속 및 라이브러리 로딩
import oracledb
import oml
import pandas as pd
import import_ipynb
from IPython.display import display
from user_functions import get_conn_cursor, get_db_version, exec_sql, dml_sql, ddl_sql, clean_text1, clean_text2

# DB 접속
user_id = 'labadmin'
pwd = 'labadmin'
dbconn = 'dbserver26ai:1521/freepdb1'

conn, cursor = get_conn_cursor(user_id,pwd,dbconn)

# DB 버전 확인
get_db_version(cursor)

/opt/jupyterhub/vector_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,PRODUCT,VERSION_FULL
0,Oracle AI Database 26ai Free,23.26.0.0.0


## 1. 하이브리드 인덱스 준비

- github 참조 : https://github.com/kairkowy/handon-labs-for-vector-search/blob/main/similarity_search/%EC%98%A4%EB%9D%BC%ED%81%B4%20%EB%B2%A1%ED%84%B0%20hybrid%20indexing%20%EC%82%AC%EC%9A%A9%20%EB%B0%A9%EB%B2%95.md

## 2.하이브리드 검색

In [2]:
#오라클 벡터 하이브리드 쿼리 샘플
hyb_query = """
SELECT jt.* 
FROM JSON_TABLE(
       json_serialize(
         dbms_hybrid_vector.search(
           json('{
             "hybrid_index_name" : "doc_hb_idx1",
             "vector" : {
               "search_text" : "' || :semantic_text || '",
               "search_mode" : "CHUNK"
             },
             "text" : {
               "contains" : "' || :contains_text || '"
             },
             "return" : {
               "topN" : 2
             }
           }')
         )
         RETURNING CLOB
       ),
       '$[*]'
       COLUMNS
         idx          FOR ORDINALITY,
         doc_rowid    VARCHAR2(30)    PATH '$.rowid',
         score        NUMBER          PATH '$.score',
         vector_score NUMBER          PATH '$.vector_score',
         text_score   NUMBER          PATH '$.text_score',
         vector_rank  NUMBER          PATH '$.vector_rank',
         text_rank    NUMBER          PATH '$.text_rank',
         chunk_text   CLOB            PATH '$.chunk_text',
         chunk_id     VARCHAR2(100)   PATH '$.chunk_id',
         paths        CLOB FORMAT JSON PATH '$.paths'
     ) jt
ORDER BY jt.idx
"""
print("하이브리드 쿼리 샘플: :" , hyb_query)

하이브리드 쿼리 샘플: : 
SELECT jt.* 
FROM JSON_TABLE(
       json_serialize(
         dbms_hybrid_vector.search(
           json('{
             "hybrid_index_name" : "doc_hb_idx1",
             "vector" : {
               "search_text" : "' || :semantic_text || '",
               "search_mode" : "CHUNK"
             },
             "text" : {
               "contains" : "' || :contains_text || '"
             },
             "return" : {
               "topN" : 2
             }
           }')
         )
         RETURNING CLOB
       ),
       '$[*]'
       COLUMNS
         idx          FOR ORDINALITY,
         doc_rowid    VARCHAR2(30)    PATH '$.rowid',
         score        NUMBER          PATH '$.score',
         vector_score NUMBER          PATH '$.vector_score',
         text_score   NUMBER          PATH '$.text_score',
         vector_rank  NUMBER          PATH '$.vector_rank',
         text_rank    NUMBER          PATH '$.text_rank',
         chunk_text   CLOB            PATH '$.chunk

In [3]:
# 조건 입력

semantic_text = input("의미검색 문장 입력 : ")   # 예: 고향사랑 기부 상반기 모금 결과
contains_text = input("키워드 검색 입력 :")  #예 : 고향사랑기부금법(and, or 등 사용 가능)

# 쿼리 실행
cursor.execute(
    hyb_query,
    semantic_text=semantic_text,
    contains_text=contains_text
)

rows = []

print("\n=== Hybrid Search Result (Text) ===\n")

for (
    idx, doc_rowid, score, vector_score, text_score,
    vector_rank, text_rank, chunk_text, chunk_id, paths
) in cursor:

    # CLOB → TEXT (1회 read)
    text = chunk_text.read() if chunk_text else ""
    # DataFrame용 row 수집
    rows.append({
        "idx": idx,
        "chunk_id": chunk_id,
        "score": score,
        "vector_score": vector_score,
        "text_score": text_score,
        "vector_rank": vector_rank,
        "text_rank": text_rank,
        "chunk_text": text
    })

# -----------------------------
# pandas DataFrame 출력
# -----------------------------
df = pd.DataFrame(rows)

# 컬럼 폭 제한 해제 (문서내용 잘림 방지)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)         

display(
    df[["chunk_id", "chunk_text"]].style.set_properties(
        subset=["chunk_text"],
        **{
            "text-align": "left",
            "white-space": "pre-wrap",
            "word-wrap": "break-word"
        }
    )
)

의미검색 문장 입력 :  고향사랑 기부 상반기 모금 결과
키워드 검색 입력 : 고향사랑기부금법



=== Hybrid Search Result (Text) ===



,chunk_id,chunk_text
0,6,"고향사랑기부 상반기 모금 실적 주요 분석 결과는 다음과 같다. ○ 월별로는 3월(약 98.2억 원, 약 8만 6천 건), 4월(약 85.9억 원, 약 6만 4천 건)에 전체 모금의 50% 이상이 집중됐다. 이는 지난 3월경 발생한 산불 피해 극복을 위한 대국민 기부 * 가 영향을 미친 것으로 보인다."
1,5,"해당하는 수준으로, 통상적으로 연말에 기부가 집중되는 점을 고려할 때 예년 모금액을 크게 넘어설 것으로 전망된다. - 2 - 〈 연도별 상반기 모금액 추이 〉 (억 원) 〈 연도별 상반기 모금건수 추이 〉 (만 건) '23.상 '24.상 '25.상 '23.상 '24.상 '25.상 □ 올해 고향사랑기부 상반기 모금 실적 주요 분석 결과는 다음과 같다."


In [4]:
# 검색 문장 입력 및 검색
pd.set_option('display.max_colwidth', 200)
print("예: 고향사랑 기부 상반기 모금 결과")
user_sentence = input("찾으실 문장을 입력하세요:")

sim_q = """
Select T.chunked_id, T.chunked_data AS 문서내용
from doc_store_chunks T, doc_store D
where D.docno = T.docno
order by vector_distance(T.embed_vector, 
                         dbms_vector.utl_to_embedding(:u_sentence, 
                               json('{"provider":"database", 
                                      "model":"MULTILINGUAL_E5_SMALL"}')),
                         cosine)
fetch first 2 rows only
"""
sim_result = exec_sql(cursor,sim_q,{"u_sentence":user_sentence})
sim_result = clean_text1(sim_result)
sim_result['문서내용']=sim_result['문서내용'].str.replace('\n', '', regex=False)
display(
    sim_result.head(10).style
    .set_properties(
           **
            {'text-align': 'left',"white-space": "pre-wrap", "word-wrap": "break-word", "max-width": "800px"})
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
)

예: 고향사랑 기부 상반기 모금 결과


찾으실 문장을 입력하세요: 고향사랑 기부 상반기 모금 결과


,CHUNKED_ID,문서내용
0,4,"○ 2025년 상반기까지 모금된 금액은 지난해11월 초까지 누적 모금액에해당하는 수준으로, 통상적으로 연말에 기부가 집중되는 점을 고려할 때예년 모금액을 크게 넘어설 것으로 전망된다.-2-〈 연도별 상반기 모금액 추이 〉(억 원)〈 연도별 상반기 모금건수 추이 〉(만 건)'23.상'24.상'25.상'23.상'24.상'25.상□ 올해 고향사랑기부 상반기 모금 실적 주요 분석 결과는 다음과 같다."
1,2,"□ 행정안전부는 2025년 고향사랑기부 상반기 모금 결과, 모금액과 모금건수가 지난 2년 같은 기간과 비교했을 때 각각 큰 폭으로 증가했다고밝혔다.○ 2025년 상반기 고향사랑기부 총 모금액은 약 348억 8천만 원, 총 모금건수는 약 27만 9천 건이다.※온라인(약 297억 원, 약 25만 7천 건) / 오프라인(약 51억 8천만 원, 약 2만 2천 건)○ 올해로 시행 3년 차를 맞이하는 고향사랑기부제는 지난 두 해 동안의"
